# US emission factors from emission factors database

In [1]:
import csv
import math
import os
import pandas as pd
from pathlib import Path
import re
import statistics

In [2]:
def separate_min_max_median(val):
    """extract value, takes median if range is given"""
    value = val.replace(' ', '').strip()
    
    range_pattern = r'(?P<min>[\d.]+)-(?P<max>[\d.]+)'
    single_pattern = r'^([\d.]+)$'

    range_match = re.search(range_pattern, value)
    single_match = re.match(single_pattern, value)

    if range_match:
        min_val = float(range_match.group('min'))
        max_val = float(range_match.group('max'))
        median = statistics.median([min_val, max_val])
        return {'value': median, 'value_min': min_val, 'value_max': max_val}
    elif single_match:
        return {'value': float(single_match.group(1)), 'value_min': None, 'value_max': None}
    else:
        return {'value': None, 'value_min': None, 'value_max': None}

    
def strip_string(value):
    """strip whitespace from string"""
    if isinstance(value, str):
        return value.strip()
    return value


def gas_name_to_formula(value, replace_dict=None):
    """replace gas name with formula"""
    if replace_dict is None:
        replace_dict = {
            'CARBON DIOXIDE': 'CO2',
            'METHANE': 'CH4',
            'NITROUS OXIDE': 'N2O'
        }
    else:
        replace_dict = {key.upper():value for key, value in replace_dict.items()}
        
    new_value = replace_dict.get(value.upper(), None)

    if new_value:
        return new_value
    
    return value


def save_to_csv(fl, data):
    """save list of dictionaries to CSV"""
    with open(fl, 'w', newline='') as csvfile:
        fieldnames = data[0].keys()  # Assuming all dictionaries have the same keys
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(data)

## 1. read and preprocess data

In [3]:
# only want to keep categories that align with GPC BASIC
IPCC_2006 = [
    '1.A - Fuel Combustion Activities',
    '1.A.1.a.ii - Combined Heat and Power Generation (CHP)',
    '1.A.3.b - Road Transportation',
    '1.B.1.a - Coal mining and handling',
    '1.B.2 - Oil and Natural Gas',
    '1.B.2.a - Oil',
    '1.B.2.b - Natural Gas',
    #'2.E.1 - Integrated Circuit or Semiconductor',
    #'2.G.1.b - Use of Electrical Equipment',
    #'3.C.7 - Rice cultivations',
    '4.B - Biological Treatment of Solid Waste',
    '4.D.1 - Domestic Wastewaster Treatment and Discharge'
]

# I only care about these columns in the dataset
COLUMNS = [
    'IPCC 2006 Source/Sink Category',
    'Value',
    'Unit',
    'Gas',
    'Fuel 2006',
    'Region / Regional Conditions',
    'Description',
    'Equation',
    'Technical Reference',
    'Parameters / Conditions',
    'Other properties',
]


df = pd.read_csv('./EFDB_USA.csv')

filt_desc = df['Description'].str.contains('emission factor', case=False, na=False)
filt_cat = df['IPCC 2006 Source/Sink Category'].isin(IPCC_2006)
filt = filt_desc & filt_cat
df_filt = df.loc[filt, COLUMNS].reset_index(drop=True)

## create list of dictionaries

In [4]:
output_list = []

# for now I am only keeping emission factors that are listed as being for the US
# for example, I am not keeping ones that are application to a list of states
# this is just for testing purposes
acceptable_region_names = ['UNITED STATES OF AMERICA', 'USA']

# loop over
for _, row in df_filt.iterrows():
    # strip extraneous whitespace
    row = row.apply(strip_string)

    # get min, max, and median value
    value = row.pop('Value')
    value_dic = separate_min_max_median(value)
    
    # rename rows and convert to dictionary
    row_dic = row.rename(
        {
            'Unit':'units', 
            'IPCC 2006 Source/Sink Category': 'ipcc_2006_category',
            'Gas': 'gas',
            'Fuel 2006': 'fuel',
            'Region / Regional Conditions': 'region',
            'Description': 'description',
            'Equation': 'equation',
            'Technical Reference': 'reference',
            'Parameters / Conditions': 'parameters',
            'Other properties': 'properties'
        }).to_dict()
    
    # merge dictionaries
    dic_tmp = {**row_dic, **value_dic}
    
    # convert nan to None
    output_dic = {key: None if (isinstance(value, float)) and math.isnan(value) else value 
                  for key, value in dic_tmp.items()}

    # replace name of gas with chemical formula
    output_dic['gas'] = gas_name_to_formula(output_dic['gas'])
    
    if output_dic['region'].upper() in acceptable_region_names:
        # change name
        output_dic['region'] = 'USA'
    
        # append to list
        output_list.append(output_dic)

## sort dictionaries by category

In [5]:
# sort list of dictionaries by ipcc category
output_list = sorted(output_list, key=lambda x: x['ipcc_2006_category'])

## save to csv

In [6]:
fl = "processed_data.csv"
save_to_csv(fl, output_list)